In [1]:
from collections import Counter
from pprint import pprint
from pathlib import Path
import pandas as pd
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx
from community import community_louvain
from helper import Preprocess
from sklearn.neighbors import NearestNeighbors
import numpy as np
from numpy import linalg as LA

from sklearn.manifold import TSNE

In [3]:
FILE_NAME = '64_relu_30_5_adjacentTF_1e-4'
DATA_DIR = Path("../../data/")

We first map the vectors in embedded space to a graph using KNN algorithm

In [1]:
emb_df = pd.read_csv(f'./emb/{FILE_NAME}.csv', index_col=0)

gm_emb_df = emb_df[emb_df['type'] == 2].drop(['type'], axis='columns')
k_emb_df = emb_df[emb_df['type'] == 1].drop(['type'], axis='columns')

gm_emb = gm_emb_df.to_numpy()
k_emb = k_emb_df.to_numpy()

# use KNN to find nearest neighbors of every node
# GM12878
idx = list(gm_emb_df.index)
idx2name = dict()
for i, name in enumerate(idx):
    idx2name[i] = name

n_neighbors = 10
print('Running KNN for GM12878...')
nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ball_tree').fit(gm_emb)
gm_distance, gm_indices = nbrs.kneighbors(gm_emb)
print('KNN for GM12878 completed!')

gm_edgelist = []
print('Creating edgelist...')
for i, name in enumerate(gm_emb_df.index):
    for j in range(len(gm_indices[i])):
        temp = []
        if j != 0:
            temp.append(name)
            temp.append(idx2name[gm_indices[i][j]])
            temp.append(gm_distance[i][j])
            gm_edgelist.append(temp)

gm_edgelist_df = pd.DataFrame(gm_edgelist)
gm_edgelist_df.columns = ['source', 'target', 'distance']

NameError: name 'pd' is not defined

In [5]:
gm_edgelist_df[gm_edgelist_df['source'] == 'MYC_gm']

,source,target,distance
522,MYC_gm,TAF1_gm,0.113848
523,MYC_gm,ZBTB40_gm,0.124476
524,MYC_gm,EGR1_gm,0.132534
525,MYC_gm,ETV6_gm,0.135190
526,MYC_gm,CHD2_gm,0.137360
527,MYC_gm,USF2_gm,0.155737
528,MYC_gm,SP1_gm,0.159003
529,MYC_gm,TARDBP_gm,0.159230
530,MYC_gm,BHLHE40_gm,0.163549


In [6]:
# K562
idx = list(k_emb_df.index)
idx2name = dict()
for i, name in enumerate(idx):
    idx2name[i] = name

print('Running KNN for K562...')
nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ball_tree').fit(k_emb)
k_distance, k_indices = nbrs.kneighbors(k_emb)
print('KNN for K562 completed!')

k_edgelist = []
print('Creating edgelist...')
for i, name in enumerate(k_emb_df.index):
    for j in range(len(k_indices[i])):
        temp = []
        if j != 0:
            temp.append(name)
            temp.append(idx2name[k_indices[i][j]])
            temp.append(k_distance[i][j])
            k_edgelist.append(temp)

k_edgelist_df = pd.DataFrame(k_edgelist)
k_edgelist_df.columns = ['source', 'target', 'distance']

Running KNN for K562...
KNN for K562 completed!
Creating edgelist...


In [30]:
k_edgelist_df

,source,target,distance
0,ZBTB33_k,NFE2_k,0.232416
1,ZBTB33_k,ZNF274_k,0.284728
2,ZBTB33_k,CBX5_k,0.284759
3,ZBTB33_k,MAZ_k,0.286061
4,ZBTB33_k,EZH2_k,0.287611
...,...,...,...
616,SMAD5_k,TBP_k,0.123027
617,SMAD5_k,PML_k,0.134127
618,SMAD5_k,JUND_k,0.136380
619,SMAD5_k,MTA2_k,0.139846
